In [3]:
# imports
# Main Goal : Having the model which summarizes any website content using llama3.2 model

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama;

In [4]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [5]:
# Define a web method which takes in the website url and parses the content with {title , text} 
# which further can be summarize through an AI model.

# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
result = Website("https://en.wikipedia.org/wiki/Artificial_intelligence")
print(f"Title: {result.title}")
print("Text:", result.text)  # Print the first 500 characters of the text

Title: Artificial intelligence - Wikipedia
Text: Jump to content
Main menu
Main menu
move to sidebar
hide
Navigation
Main page
Contents
Current events
Random article
About Wikipedia
Contact us
Contribute
Help
Learn to edit
Community portal
Recent changes
Upload file
Special pages
Search
Search
Appearance
Donate
Create account
Log in
Personal tools
Donate
Create account
Log in
Pages for logged out editors
learn more
Contributions
Talk
Contents
move to sidebar
hide
(Top)
1
Goals
Toggle Goals subsection
1.1
Reasoning and problem-solving
1.2
Knowledge representation
1.3
Planning and decision-making
1.4
Learning
1.5
Natural language processing
1.6
Perception
1.7
Social intelligence
1.8
General intelligence
2
Techniques
Toggle Techniques subsection
2.1
Search and optimization
2.1.1
State space search
2.1.2
Local search
2.2
Logic
2.3
Probabilistic methods for uncertain reasoning
2.4
Classifiers and statistical learning methods
2.5
Artificial neural networks
2.6
Deep learning
2.7
GPT
2.8
Hardw

In [7]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
# A function that writes a User Prompt that asks for summaries of websites:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
# See how this function creates exactly the format above
# We ensure that we pass in the custom system prompt + user prompt. 
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [12]:
# And now: call the OpenAI API. You will get very familiar with this!
# ChatGPT-4o models return a markdown string which further can be rendered nicely in Jupyter
def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response['message']['content'];

In [13]:
def displaySummarize(url):
    summary = summarize(url)
    display(Markdown(summary))

In [14]:
displaySummarize("https://anthropic.com");

# Anthropic Overview
Anthropic is an organization focused on building AI to serve humanity's long-term well-being. Their flagship product, Claude, is a large language model designed to handle complex tasks and enable the creation of custom AI-powered applications.

## Key Features and Products

*   **Claude Opus 4.1**: The latest AI model from Anthropic, offering improved performance and capabilities.
*   **Claude Sonnet 4** and **Claude Haiku 3.5**: Other powerful models in the Claude family, each with its unique strengths and applications.
*   **API Platform**: A suite of APIs enabling developers to build custom experiences with Claude.

## Research and Views on AI Safety

Anthropic prioritizes responsible AI development, focusing on human benefit at the core of their work. Their research encompasses various aspects of AI safety, including:

*   **Core Views on AI Safety**
*   **Anthropic's Responsible Scaling Policy**

## Partnerships and Integrations

Anthropic partners with major cloud providers to bring Claude to a wider audience, including:

*   **Claude in Amazon Bedrock**
*   **Claude on Google Cloud’s Vertex AI**

## News and Announcements

Recent updates from Anthropic include:

*   **Project Vend**: A new initiative addressing agentic misalignment.
*   **Tracing the thoughts of a large language model**: Research into interpretability.

## Career Opportunities

Anthropic offers various roles for those interested in contributing to their mission, including:

*   **See open roles**

## Additional Resources

For further learning and exploration, Anthropic provides:

*   **Anthropic Academy**
*   **Customer stories**
*   **Engineering at Anthropic**